In [7]:
!pip install torch torchvision torchaudio
!pip install git+https://github.com/ultralytics/yolov5.git
!pip install gradio

Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 5, in <module>
    from pip._internal.cli.main import main
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main.py", line 11, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_runnable_pip
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/build_env.py", line 19, in <module>
    from pip._internal.cli.spinners import open_spinner
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/spinners.py", line 9, in <module>
    from pip._internal.utils.logging import get_indentation
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/util

In [ ]:
import cv2  # مكتبة للتعامل مع الفيديو والصور
import torch  # مكتبة للتعلم العميق
from PIL import Image, ImageDraw  # مكتبة لتحرير الصور
import gradio as gr  # مكتبة لإنشاء واجهات تفاعلية
import pandas as pd  # مكتبة للتعامل مع البيانات
from transformers import pipeline  # مكتبة للترجمة

# تحميل نموذج YOLOv5
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# تحميل نموذج الترجمة
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-tc-big-en-ar")

# دالة لاكتشاف الكائنات ورسم الإطارات في الصور
def detect_and_draw_image(input_image):
    results = model(input_image)
    detections = results.xyxy[0].numpy()

    draw = ImageDraw.Draw(input_image)

    counts = {}
    for detection in detections:
        xmin, ymin, xmax, ymax, conf, class_id = detection
        label = model.names[int(class_id)]
        counts[label] = counts.get(label, 0) + 1

        draw.rectangle([(xmin, ymin), (xmax, ymax)], outline="red", width=2)
        draw.text((xmin, ymin), f"{label}: {conf:.2f}", fill="white")

    # ترجمة التسميات إلى العربية
    translated_labels = translator(list(counts.keys()))

    # إنشاء DataFrame لتخزين النتائج
    df = pd.DataFrame({
        'Label (English)': list(counts.keys()),
        'Label (Arabic)': [t['translation_text'] for t in translated_labels],
        'Object Count': list(counts.values())
    })

    return input_image, df

# دالة لاكتشاف الكائنات ورسم الإطارات في الفيديو
def detect_and_draw_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    overall_counts = {}

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(frame, (640, 480))
        results = model(frame)
        detections = results.xyxy[0].numpy()

        for detection in detections:
            xmin, ymin, xmax, ymax, conf, class_id = detection
            label = model.names[int(class_id)]
            overall_counts[label] = overall_counts.get(label, 0) + 1

            cv2.rectangle(frame, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (255, 0, 0), 2)
            cv2.putText(frame, f"{label}: {conf:.2f}", (int(xmin), int(ymin) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        frames.append(frame)

    cap.release()

    output_path = 'output.mp4'
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), 20.0, (640, 480))

    for frame in frames:
        out.write(frame)
    out.release()

    # ترجمة التسميات إلى العربية
    translated_labels = translator(list(overall_counts.keys()))

    # إنشاء DataFrame لتخزين النتائج
    df = pd.DataFrame({
        'Label (English)': list(overall_counts.keys()),
        'Label (Arabic)': [t['translation_text'] for t in translated_labels],
        'Object Count': list(overall_counts.values())
    })

    return output_path, df

# إنشاء واجهات لتطبيق Gradio
image_interface = gr.Interface(
    fn=detect_and_draw_image,
    inputs=gr.Image(type="pil", label="Upload Image"),
    outputs=[gr.Image(type="pil"), gr.Dataframe(label="Object Counts")],
    title="Object Detection for Images",
    description="Upload an image to see the objects detected by YOLOv5 with bounding boxes and their counts.",
    examples=['/content/assets/12222-1170x780.jpg', '/content/assets/0.05 DOT 2016 OPI After 1st Ave 47 to 48 and 55 to 59 Manhattan 002_0.webp']
)

video_interface = gr.Interface(
    fn=detect_and_draw_video,
    inputs=gr.Video(label="Upload Video"),
    outputs=[gr.Video(label="Processed Video"), gr.Dataframe(label="Object Counts")],
    title="Object Detection for Videos",
    description="Upload a video to see the objects detected by YOLOv5 with bounding boxes and their counts."
)

# دمج الواجهات في تطبيق واحد
app = gr.TabbedInterface([image_interface, video_interface], ["Image Detection", "Video Detection"])

# تشغيل التطبيق
app.launch(debug=True)


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-10-1 Python-3.10.12 torch-2.4.1+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://94d185d10d6f0ccaa8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
